In [ ]:
!python --version
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xvzf spark-3.3.0-bin-hadoop3.tgz
!ls /content/spark-3.3.0-bin-hadoop3
!pip install findspark
import os
os.environ['Spark_Home'] = '/content/spark-3.3.0-bin-hadoop3'
import findspark
findspark.init
from google.colab import drive 
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install cartopy
import cartopy
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySpark 3.3 on google colab').getOrCreate()

Python 3.8.15
--2022-12-02 07:10:49--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-12-02 07:10:50 ERROR 404: Not Found.

tar (child): spark-3.3.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
ls: cannot access '/content/spark-3.3.0-bin-hadoop3': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 124015 files and directories currently installed.)
Preparing to u

In [ ]:
import time
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, \
                                      NaiveBayes, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import regexp_replace, col

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
dataset = spark.read.csv("/content/drive/MyDrive/BigData&IoT/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv", inferSchema=True, header=True, multiLine=True,
                         ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

In [ ]:
print("Dataset schema:")
dataset.printSchema()

Dataset schema:
root
 |-- Destination Port: integer (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Total Fwd Packets: integer (nullable = true)
 |-- Total Backward Packets: integer (nullable = true)
 |-- Total Length of Fwd Packets: integer (nullable = true)
 |-- Total Length of Bwd Packets: integer (nullable = true)
 |-- Fwd Packet Length Max: integer (nullable = true)
 |-- Fwd Packet Length Min: integer (nullable = true)
 |-- Fwd Packet Length Mean: double (nullable = true)
 |-- Fwd Packet Length Std: double (nullable = true)
 |-- Bwd Packet Length Max: integer (nullable = true)
 |-- Bwd Packet Length Min: integer (nullable = true)
 |-- Bwd Packet Length Mean: double (nullable = true)
 |-- Bwd Packet Length Std: double (nullable = true)
 |-- Flow Bytes/s: double (nullable = true)
 |-- Flow Packets/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: integer (nullable = true)
 |-- 

In [ ]:
print("Original dataset sizes: {row} samples, {cols} features".format(row=dataset.count(), cols=len(dataset.columns)))
# replace 'REPLACEMENT CHARACTER' Unicode Character (�) u"\uFFFD" in some Label values
dataset = dataset.withColumn("Label", regexp_replace("Label", u"\uFFFD ", ""))
dataset.select("Label").groupBy("Label").count().orderBy("count", ascending=False).show()

dataset = dataset.where(col('Flow Duration') > 0) \
                 .where(col('Init_Win_bytes_forward') > 0) \
                 .where(col('Init_Win_bytes_backward') > 0) \
                 .where(col('Flow IAT Min') > 0) \
                 .where(col('Fwd IAT Min') > 0) \
                 .where(col('Fwd IAT Max') > 0)

print("\nParsed dataset sizes: {row} samples, {cols} features".format(row=dataset.count(), cols=len(dataset.columns)))
dataset.select("Label").groupBy("Label").count().orderBy("count", ascending=False).show()


Original dataset sizes: 529918 samples, 79 features
+------+------+
| Label| count|
+------+------+
|BENIGN|529918|
+------+------+


Parsed dataset sizes: 142031 samples, 79 features
+------+------+
| Label| count|
+------+------+
|BENIGN|142031|
+------+------+



In [ ]:
spark = SparkSession.builder.appName("Network Attacks Classifier CICIDS2017").master("local").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

start = time.time()

features = [f for f in dataset.columns if f not in ["Label"]]
df_assembler = VectorAssembler(inputCols=features, outputCol="features").setHandleInvalid("skip")
dataset = df_assembler.transform(dataset)
# dataset.printSchema()

label_indexer = StringIndexer(inputCol="Label", outputCol="Label_Idx").setHandleInvalid("skip").fit(dataset)
dataset = label_indexer.transform(dataset)
# dataset.printSchema()

# dataset.select(["Label", "Label_Idx"]).distinct().orderBy("Label_Idx").show()
label_list = dataset.select(["Label", "Label_Idx"]).distinct().orderBy("Label_Idx").select("Label").rdd.flatMap(lambda x: x).collect()
# print(label_list)

dataset = dataset.select(["features","Label_Idx"])
# dataset.printSchema()

train_set, test_set = dataset.randomSplit([0.75, 0.25], seed=2019)
print("Training set Count: " + str(train_set.count()))
print("Test set Count: " + str(test_set.count()))

# Logistic Regression model
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0.8, featuresCol="features",
                        labelCol="Label_Idx", family="multinomial")

# Decision Tree model
dt = DecisionTreeClassifier(labelCol="Label_Idx", featuresCol="features", maxBins=len(features))

# Random Forest model
rf = RandomForestClassifier(labelCol="Label_Idx", featuresCol="features", numTrees=20, maxBins=len(features))

# Naive Bayes Multinomial
nb = NaiveBayes(labelCol="Label_Idx", featuresCol="features", smoothing=1.0, modelType="multinomial")

classifiers = {"Logistic Regression": lr, "Decision Tree": dt,
               "Random Forest": rf, "Naive Bayes Multinomial": nb}

metrics = ["accuracy", "weightedPrecision", "weightedRecall", "f1"]

print("\nModels Evaluation:")
print("{:-<24}".format(""))
for idx, c in enumerate(classifiers):
	print(c)
	# fit the model
	model = classifiers[c].fit(train_set)
	
	# make predictions
	predictions = model.transform(test_set)
	predictions.cache()
	
	# evaluate performance
	evaluator = MulticlassClassificationEvaluator(labelCol="Label_Idx", predictionCol="prediction")
	
	for m in metrics:
		evaluator.setMetricName(m)
		metric = evaluator.evaluate(predictions)
		print("{name} = {value:.2f}".format(name=m, value=metric))

	print("{:-<24}".format(""))

stop = time.time()
print("\nRunning time for Spark job '{name}': {time:.2f} s"
      .format(name=spark.conf.get("spark.app.name"), time=(stop-start)))

plt.show()


Training set Count: 106507
Test set Count: 35524

Models Evaluation:
------------------------
Logistic Regression
accuracy = 1.00
weightedPrecision = 1.00
weightedRecall = 1.00
f1 = 1.00
------------------------
Decision Tree
accuracy = 1.00
weightedPrecision = 1.00
weightedRecall = 1.00
f1 = 1.00
------------------------
Random Forest
accuracy = 1.00
weightedPrecision = 1.00
weightedRecall = 1.00
f1 = 1.00
------------------------
Naive Bayes Multinomial
accuracy = 1.00
weightedPrecision = 1.00
weightedRecall = 1.00
f1 = 1.00
------------------------

Running time for Spark job 'Network Attacks Classifier CICIDS2017': 208.71 s
